## feauterization

Lo que hago es feauterizo, normalizo y despues lo mando como un json para poder hacerlo csv y despues importarlo facil


### Como es la normalizacion que estoy usando?

Bueno lo que hago es hacer que todos las features sean normalizadas entre 0 1, para que despues durante el entrenamiento cuando se concatene al output de las convoluciones se escale a sus respectivos maximos y minimos, de cada compuesto.

Esta idea la saco de como funciona el minmax scaleing
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

```python
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # esto lo hago afuera, pq siempre es lo mismo para cada vez que se usa el vector de features
X_scaled = X_std * (max - min) + min # este es el paso que hago adentro de la red, en base a cada output de las covoluciones
```

Se hace en dos pasos, ese Scaleing: primero se genera que todo el vector este entre 01 y despues se le improntan los maximos y minimos de los outputs de las convolucinoes.

In [3]:
from deepchem.feat import RDKitDescriptors
import pandas as pd
import numpy as np
import json
import os

def add_column_feturized_json(path_original,output_path):
    namefile = path_original.split("/")[-1]
    name = namefile[:namefile.find(".csv")]
    featurizer = RDKitDescriptors()     
    df = pd.read_csv(path_original)
    rdkit_features_not_std = featurizer.featurize(df["smiles"])
    remove_nan = ~((np.isnan(rdkit_features_not_std)).any(axis=1))
    rdkit_features_not_std = rdkit_features_not_std[remove_nan]
    df_output = df[remove_nan] 
    comps_droped = len(df) - len(rdkit_features_not_std)
    print("problematic compounds droped:",comps_droped)
    rdkit_features_not_std_max = rdkit_features_not_std.max(axis=0,keepdims=True)
    rdkit_features_not_std_min = rdkit_features_not_std.min(axis=0,keepdims=True)
    rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)
    # ESTO ES UNA INCOSISTENCIA BRUTAL PORQUE VAN A HABER DISTINTAS FEATURES EN CADA TARGET...
    rdkit_features = rdkit_features[:,(~np.isnan(rdkit_features)).all(axis=0)]
    
    np.save(os.path.join(output_path,(name+"features_rdkit.npy")),rdkit_features)
    df_output.to_csv(os.path.join(output_path,(name+"features_rdkit.csv")),index=False)


In [4]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL262.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 4


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


In [5]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL4657.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 0


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


In [12]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL4072.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 1


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


In [13]:
add_column_feturized_json("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2850.csv","/home/sjinich/disco/TrypanoDEEPscreen/.data/processed")

problematic compounds droped: 0


/tmp/ipykernel_202250/1431919571.py:20: RuntimeWarning: invalid value encountered in true_divide
  rdkit_features = (rdkit_features_not_std - rdkit_features_not_std_min) / (rdkit_features_not_std_max - rdkit_features_not_std_min)


trials de splting

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2850features_rdkit.csv")
features = np.load("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2850features_rdkit.npy")

In [6]:
len(features[0])

193

In [3]:
features_1 = np.load("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL262features_rdkit.npy")

In [5]:
len(features_1[0])

198

In [7]:
features_2 = np.load("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL4072features_rdkit.npy")

In [8]:
len(features_2[0])

195